# xorxorxor (Writeup) [ESP]
###### _Categoría: Crypto_  
###### _Dificultad: Fácil_
###### _by: D-Cryp7_

Descripción:
* Who needs AES when you have XOR?

Se nos entrega el siguiente código
```python
#!/usr/bin/python3
import os
flag = open('flag.txt', 'r').read().strip().encode()

class XOR:
    def __init__(self):
        self.key = os.urandom(4)
    def encrypt(self, data: bytes) -> bytes:
        xored = b''
        for i in range(len(data)):
            xored += bytes([data[i] ^ self.key[i % len(self.key)]])
        return xored
    def decrypt(self, data: bytes) -> bytes:
        return self.encrypt(data)

def main():
    global flag
    crypto = XOR()
    print ('Flag:', crypto.encrypt(flag).hex())

if __name__ == '__main__':
    main()
```
y la flag encriptada


In [1]:
c = '134af6e1297bc4a96f6a87fe046684e8047084ee046d84c5282dd7ef292dc9'

Analizando el código, nos damos cuenta de que la función de cifrado realiza un XOR (operador `^`) entre cada byte de la flag con cada byte de la llave. Si la longitud de la llave fuera equivalente a la de la flag, este sistema sería del estilo OTP (One Time Pad) y sería más complicado de resolver (imposible si el generador de llaves fuera un TRNG). En el caso de este challenge, dado que la longitud de la llave es de 4 bytes, es evidente que no es lo suficientemente larga como para cifrar toda la flag, por lo que la llave se reutiliza. Este es el problema que hemos identificado, y lo explotaremos para obtener el mensaje original. Consideremos un sistema de ecuaciones que ilustra el proceso de cifrado:
\begin{equation}
  c_0 = m_0 \oplus k_0
\end{equation}
\begin{equation}
  c_1 = m_1 \oplus k_1
\end{equation}
\begin{equation}
  c_2 = m_2 \oplus k_2
\end{equation}
\begin{equation}
  c_3 = m_3 \oplus k_3
\end{equation}
\begin{equation}
  c_4 = m_4 \oplus k_0
\end{equation}
\begin{equation}
  c_5 = m_5 \oplus k_1
\end{equation}
\begin{equation}
  \vdots
\end{equation}
y así sucesivamente, donde $c$, $m$ y $k$ son el conjunto de cada uno de los bytes del mensaje cifrado, la flag y la llave, respectivamente. Si podemos resolver las 4 primeras ecuaciones, podemos recuperar la llave. Casualmente, conocemos los 4 primeros bytes de la flag (`HTB{`), y los utilizaremos para hacer el siguiente sistema de ecuaciones (2 celdas más abajo)


In [2]:
# obtenemos los primeros 4 bytes del mensaje cifrado
c_bytes = bytes.fromhex(c)
for i in range(4):
  print(c_bytes[i])
c_bytes

19
74
246
225


b'\x13J\xf6\xe1){\xc4\xa9oj\x87\xfe\x04f\x84\xe8\x04p\x84\xee\x04m\x84\xc5(-\xd7\xef)-\xc9'

In [3]:
# obtenemos los 4 bytes que conocemos de la flag
kpt = b'HTB{'
for i in kpt:
  print(i)
kpt

72
84
66
123


b'HTB{'

\begin{equation}
  19 = 72 \oplus k_0
\end{equation}
\begin{equation}
  74 = 84 \oplus k_1
\end{equation}
\begin{equation}
  246 = 66 \oplus k_2
\end{equation}
\begin{equation}
  225 = 123 \oplus k_3
\end{equation}
Así pues, podemos obtener cada valor de $k_i \in \{0, 255\}$ aplicando el XOR a ambos lados cada ecuación con los respectivos bytes que conocemos de la flag, ya que (si quisiéramos despejar $z$):
\begin{equation}
  x = y \oplus z \text{ / } \oplus y
\end{equation}
\begin{equation}
  x \oplus y = y \oplus z \oplus y
\end{equation}
\begin{equation}
  x \oplus y = z \oplus y \oplus y \text{ (conmutatividad de la función XOR)}
\end{equation}
\begin{equation}
  x \oplus y = z \oplus 0  \text{ (el XOR entre valores iguales es 0)}
\end{equation}
\begin{equation}
  x \oplus y = z  \text{ (el XOR entre 0 y x es x)}
\end{equation}
entonces:
\begin{equation}
  k_0 = 19 \oplus 72
\end{equation}
\begin{equation}
  k_1 = 74 \oplus 84
\end{equation}
\begin{equation}
  k_2 = 246 \oplus 66
\end{equation}
\begin{equation}
  k_3 = 225 \oplus 123
\end{equation}
Finalmente, de aquí en adelante obtenemos la flag directamente, ya que por lo explicado anteriormente, el descifrar con XOR coincide con el cifrado

In [4]:
k = []
k.append(19 ^ 72)
k.append(74 ^ 84)
k.append(246 ^ 66)
k.append(225 ^ 123)
key = bytes(bytearray(k))
key

b'[\x1e\xb4\x9a'

In [5]:
import os

class XOR:
    def __init__(self):
        self.key = os.urandom(4)
    def encrypt(self, data: bytes, key) -> bytes:
        xored = b''
        for i in range(len(data)):
            xored += bytes([data[i] ^ key[i % len(key)]])
        return xored, key
    def decrypt(self, data: bytes, key) -> bytes:
        return self.encrypt(data, key)

crypto = XOR()
c = crypto.decrypt(c_bytes, key)
print('Flag:', c[0])

Flag: b'HTB{rep34t3d_x0r_n0t_s0_s3cur3}'
